* GUI For Lung_Disease_Diagnosis
>Git hub :https://github.com/Souhrdya-308/Lung_Disease_Dignosis



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:

!pip install -q gradio
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import os


MODEL_PATH = "/content/drive/MyDrive/Lung_Model/model_1.keras"
CLASS_NAMES = [ "Normal","Tuberculosis","Pneumonia"]
IMAGE_SIZE = (224, 224)
MODEL_ACCURACY = 0.94


if not os.path.exists(MODEL_PATH):
    raise FileNotFoundError("Model file not found at path!")

model = tf.keras.models.load_model(MODEL_PATH)
print("Model loaded successfully!")



def img_pre_process(pil_img):
    """
    Converts PIL image to model-ready tensor.
    - resize
    - convert to RGB
    - normalize to [0,1]
    - return numpy array (H,W,3)
    """
    pil_img = pil_img.convert("RGB")
    pil_img = pil_img.resize(IMAGE_SIZE)
    arr = np.array(pil_img).astype("float32") / 255.0
    return arr


INFO = {
    "Normal": (
        "🫁 **Healthy Lungs**\n\n"
        "Your chest X-ray shows no visible infection.\n\n"
        "General health tips:\n"
        "- Stay hydrated\n"
        "- Avoid smoking\n"
        "- Maintain good air quality\n"
        "- Exercise regularly\n"
    ),

    "Pneumonia": (
        "⚠️ **Pneumonia Detected (Possible)**\n\n"
        "Pneumonia is an infection causing inflammation in lung air sacs.\n\n"
        "Symptoms include:\n"
        "- Fever / chills\n"
        "- Cough with or without mucus\n"
        "- Chest pain\n"
        "- Difficulty breathing\n"
        "- Fatigue and weakness\n\n"
        "Complications:\n"
        "- Fluid in lungs (pleural effusion)\n"
        "- Sepsis (life-threatening)\n\n"
        "Urgency: Seek medical evaluation within **24–48 hours**."
    ),

    "Tuberculosis": (
        "⚠️ **Tuberculosis (TB) Suspected**\n\n"
        "TB is a bacterial infection caused by *Mycobacterium tuberculosis*.\n\n"
        "Symptoms include:\n"
        "- Persistent cough (>2 weeks)\n"
        "- Coughing blood\n"
        "- Night sweats\n"
        "- Weight loss\n"
        "- Fever\n\n"
        "Urgency: Visit a TB clinic immediately for sputum / GeneXpert testing."
    )
}



def analyze_uploaded(pil_img):
    if pil_img is None:
        return None, None, None, "Please upload an image."

    img = img_pre_process(pil_img)
    preds = model.predict(tf.expand_dims(img, axis=0))[0]

    # softmax (if needed)
    if preds.max() > 1 or preds.min() < 0:
        preds = tf.nn.softmax(preds).numpy()

    idx = int(np.argmax(preds))
    predicted_class = CLASS_NAMES[idx]

    probabilities = {CLASS_NAMES[i]: float(preds[i]) for i in range(len(CLASS_NAMES))}
    info_text = INFO[predicted_class]

    return predicted_class, probabilities, pil_img, info_text



def analyze_url(url):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        return analyze_uploaded(img)
    except:
        return None, None, None, "Could not load image from URL."



with gr.Blocks() as demo:

    gr.Markdown("# 🫁 Lung Disease Classifier")
    gr.Markdown(f"**Model Validation Accuracy:** `{MODEL_ACCURACY*100:.2f}%`\n")
    gr.Markdown("Upload a chest X-ray or paste an image URL.")

    with gr.Row():
        with gr.Column(scale=5):
            with gr.Tab("Upload Image"):
                upload_img = gr.Image(type="pil", label="Upload Chest X-ray")
                upload_button = gr.Button("Analyze Image")

            with gr.Tab("Image URL"):
                url_input = gr.Textbox(label="Image URL")
                url_button = gr.Button("Analyze URL")

        with gr.Column(scale=5):
            pred_output = gr.Textbox(label="Predicted Class", interactive=False)
            prob_output = gr.Label(label="Prediction Probabilities")
            img_preview = gr.Image(type="pil", label="X-ray Preview")
            info_output = gr.Markdown()

    upload_button.click(analyze_uploaded, upload_img,
                        [pred_output, prob_output, img_preview, info_output])

    url_button.click(analyze_url, url_input,
                     [pred_output, prob_output, img_preview, info_output])

demo.launch(inbrowser=True, share=True)


Model loaded successfully!
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://094472426c9b5b3cd7.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
